In [795]:
from IPython.display import Image
from IPython.core.display import HTML 
#from __future__ import print_func  tion, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import json
from sklearn.model_selection import train_test_split
Image(url= "https://cdn-images-1.medium.com/max/1600/1*UkI9za9zTR-HL8uM15Wmzw.png")

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 500)


In [796]:
# made in 1.9.0
tf.__version__

'1.9.0'

In [825]:
# the columns use for predictions
# NOTE:
# When adding more features
# make sure make the model in a differnt path because it screws with the checkpoints
saved_model = "./6_col_5_gesture_checkpoints_1/"
features = ['posX', 'posY', 'posZ', 'velX', 'velY', 'velZ']
col_numbers = len(features)


In [799]:
# copying the dataframe so I have a before and after
df_cir_hor_cclk_shape = pd.read_csv('./Data/extracted_data/cir_hor_cclk.csv') # saving circle horizontal counter clockwise csv as df
df_v_shape = pd.read_csv('./Data/extracted_data/v_shape.csv')
df_x_shape = pd.read_csv('./Data/extracted_data/x_shape.csv')
df_twist_cclk_shape = pd.read_csv('./Data/extracted_data/twist_cclk.csv')
df_cir_ver_cclk_shape = pd.read_csv('./Data/extracted_data/cir_ver_cclk.csv')

df_cir_cclk = df_cir_hor_cclk_shape.copy()
df_v = df_v_shape.copy()
df_x = df_x_shape.copy()
df_twist_cclk = df_twist_cclk_shape.copy()
df_cir_ver_cclk = df_cir_ver_cclk_shape.copy()

number_classes = 5  # this is the value of gestures to be recognised // the number of dataframe being constructed

# to see all the headers/columns of the dataframe
# every column can be used as a feature data set except for tester and trial
# which are markings cut off points in the data
list(df_cir_cclk)




['tester',
 'trial',
 'rightHanded',
 'timestamp',
 'posX',
 'posY',
 'posZ',
 'posRelX',
 'posRelY',
 'posRelZ',
 'velX',
 'velY',
 'velZ',
 'quatW',
 'quatX',
 'quatY',
 'quatZ',
 'accRelX',
 'accRelY',
 'accRelZ',
 'accAbsX',
 'accAbsY',
 'accAbsZ']

In [801]:
def pad_features(not_pad_features):
    """
    A function that returns a padded array with a size 300
    :param not_pad_features: 
    :return: an np.array of padded vector with the length of  300 
    """
    
    list_pad_vec = list()
    feature_len = [len(s) for s in not_pad_features]
    if max(feature_len) > 300:
        raise Exception('the longest stroke is greater than 300. len: [}'
                        .format(max(feature_len)))
    padding = np.zeros((300, col_numbers))
    
    for x in not_pad_features:
        padding[:x.shape[0], :x.shape[1]] = x 
        list_pad_vec.append(padding)
        padding = np.zeros((300, col_numbers))
        
    return(np.array(list_pad_vec))

In [802]:
def group_drawn_shape(grouped):
    """
    
    :param grouped: grouped value from the dataframe, ['trial'. 'tester']
    :return: np.array of grouped vectors
    """
    source = [x for x in grouped]
    list_vec = list()
    for v in source:
        df = v[1]
        df = df[features]
        x = df.values
        list_vec.append(x)
    return np.array(list_vec)

In [803]:
# group data using trial and tester
cir_cclk_features = group_drawn_shape(df_cir_cclk.groupby(['trial', 'tester']))
v_features = group_drawn_shape(df_v.groupby(['trial', 'tester']))
x_shape_features = group_drawn_shape(df_x.groupby(['trial', 'tester']))
twist_cclk_features = group_drawn_shape(df_twist_cclk.groupby(['trial', 'tester']))
cir_ver_cclk_features = group_drawn_shape(df_cir_ver_cclk.groupby(['trial', 'tester']))


# pad the feautre columns with a max height/row of 300 with zeros
pad_cir_cclk_features = pad_features(cir_cclk_features)
pad_v_features = pad_features(v_features)
pad_x_features = pad_features(x_shape_features)
pad_cclk_twist = pad_features(twist_cclk_features)
pad_cir_ver_cclk = pad_features(cir_ver_cclk_features)

In [805]:
print("ccl horizontal features shape: \t", pad_cir_cclk_features.shape)
print("v features shape: \t \t", pad_v_features.shape)
print("x features shape: \t \t", pad_x_features.shape)
print("cclk twist features shape: \t \t", pad_cclk_twist.shape)
print("circle vertical cclk shape: \t \t", pad_cir_ver_cclk.shape)

#combine the padded data so we have one data to work with
data = np.vstack((pad_cir_cclk_features, pad_v_features))
data = np.vstack((data, pad_x_features))
data = np.vstack((data, pad_cclk_twist))
data = np.vstack((data,pad_cir_ver_cclk))
data.shape



ccl horizontal features shape: 	 (280, 300, 6)
v features shape: 	 	 (280, 300, 6)
x features shape: 	 	 (280, 300, 6)
cclk twist features shape: 	 	 (281, 300, 6)
circle vertical cclk shape: 	 	 (281, 300, 6)


(1402, 300, 6)

In [812]:
horizontal_circle_label = np.zeros(280)
v_shape_label = np.zeros(280)
x_shape_label = np.zeros(280)
twist_cclk_label = np.zeros(281)
cir_ver_cclk_label = np.zeros(281)

v_shape_label[:] = 1
x_shape_label[:] = 2
twist_cclk_label[:] = 3
cir_ver_cclk_label[:] = 4

labels = np.concatenate([horizontal_circle_label, v_shape_label])
labels = np.concatenate([labels, x_shape_label])
labels = np.concatenate([labels, twist_cclk_label])
labels = np.concatenate([labels, cir_ver_cclk_label])

print(labels.shape)
print(labels[275:285])
print(labels[555:565])
print(labels[835:845])
print(labels[1116:1126])



(1402,)
[0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 2. 2. 2. 2. 2.]
[2. 2. 2. 2. 2. 3. 3. 3. 3. 3.]
[3. 3. 3. 3. 3. 4. 4. 4. 4. 4.]


In [813]:
# split the data to training training and testing data.
# x = features
# y = labels
x_train, x_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.10, random_state=42)

print(x_train.shape)
print(y_train.shape)

(1261, 300, 6)
(1261,)


In [814]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x": x_train},
    y = np.int32(y_train),
    num_epochs=1,
    shuffle=False
)

In [815]:
train_input_fn

<function tensorflow.python.estimator.inputs.numpy_io.numpy_input_fn.<locals>.input_fn>

In [816]:
train_input_fn()

({'x': <tf.Tensor 'fifo_queue_DequeueUpTo_17:1' shape=(?, 300, 6) dtype=float64>},
 <tf.Tensor 'fifo_queue_DequeueUpTo_17:2' shape=(?,) dtype=int32>)

In [839]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=np.int32(y_test),
    num_epochs=25,
    shuffle=False
)

In [840]:
some_gesture =x_test[0:20]
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": some_gesture},
    num_epochs=1,
    shuffle=False
)

In [841]:
some_gesture_labels = y_test[0:20]

In [842]:
# slices the gesture features of height of 300 and col_numbers 
features_x = tf.feature_column.numeric_column(key="x", shape=(300, col_numbers))
feature_columns = [features_x]

In [844]:
# IGNORE FOR NOW
# this model is using essentially 1 giant feature (posX, posY, posZ)

#keys = ['posX', 'posY', 'posZ']
#feature_columns = list()
#for k in keys:
#    feature_columns.append(tf.feature_column.numeric_column(key=k))


#feature_columns[0]

In [845]:
num_hidden_units = [512, 256, 128]

In [846]:
model = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                   hidden_units=num_hidden_units,
                                   activation_fn=tf.nn.relu,
                                   n_classes=number_classes,
                                   model_dir=saved_model)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000026E36C25400>, '_save_checkpoints_secs': 600, '_task_type': 'worker', '_is_chief': True, '_keep_checkpoint_max': 5, '_train_distribute': None, '_evaluation_master': '', '_service': None, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_device_fn': None, '_task_id': 0, '_global_id_in_cluster': 0, '_save_summary_steps': 100, '_master': '', '_num_worker_replicas': 1, '_tf_random_seed': None, '_session_config': None, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './6_col_5_gesture_checkpoints_1/'}


In [847]:
model.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Restoring parameters from ./6_col_5_gesture_checkpoints_1/model.ckpt-10


...

INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Saving checkpoints for 10 into ./6_col_5_gesture_checkpoints_1/model.ckpt.


INFO:tensorflow:loss = 119.371346, step = 11


INFO:tensorflow:Saving checkpoints for 20 into ./6_col_5_gesture_checkpoints_1/model.ckpt.


INFO:tensorflow:Loss for final step: 37.526356.


In [848]:
result = model.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2018-11-13-03:11:00


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Restoring parameters from ./6_col_5_gesture_checkpoints_1/model.ckpt-20


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Finished evaluation at 2018-11-13-03:11:01


INFO:tensorflow:Saving dict for global step 20: accuracy = 0.9007092, average_loss = 0.38165295, global_step = 20, loss = 48.047382


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20: ./6_col_5_gesture_checkpoints_1/model.ckpt-20


In [849]:
result

{'accuracy': 0.9007092,
 'average_loss': 0.38165295,
 'global_step': 20,
 'loss': 48.047382}

In [850]:
print("Classification Accuracy for data set (posX, posY, posZ): "
      "{0:.2%}".format(result["accuracy"]))

Classification Accuracy for data set (posX, posY, posZ): 90.07%


In [852]:
# predictions!

predictions = model.predict(input_fn=predict_input_fn)


In [853]:
cls = [p['classes'] for p in predictions]
cls

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Restoring parameters from ./6_col_5_gesture_checkpoints_1/model.ckpt-20


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


[array([b'3'], dtype=object),
 array([b'3'], dtype=object),
 array([b'0'], dtype=object),
 array([b'4'], dtype=object),
 array([b'2'], dtype=object),
 array([b'1'], dtype=object),
 array([b'3'], dtype=object),
 array([b'3'], dtype=object),
 array([b'1'], dtype=object),
 array([b'1'], dtype=object),
 array([b'0'], dtype=object),
 array([b'2'], dtype=object),
 array([b'1'], dtype=object),
 array([b'2'], dtype=object),
 array([b'0'], dtype=object),
 array([b'0'], dtype=object),
 array([b'1'], dtype=object),
 array([b'2'], dtype=object),
 array([b'1'], dtype=object),
 array([b'4'], dtype=object)]

In [854]:
cls_pred = np.array(cls, dtype='int').squeeze()
cls_pred

array([3, 3, 0, 4, 2, 1, 3, 3, 1, 1, 0, 2, 1, 2, 0, 0, 1, 2, 1, 4])

In [855]:
some_gesture  # gestures to be predicted
print("the prediction is: \t ",cls_pred)      # predicted gestures
print("the actual is: \t\t ",np.int32(some_gesture_labels))

print('\n')

print(
    "0: \t Circle Horizontal Counter Clockwise Gesture \n"
    "1: \t V Shape Gesture \n"
    "2: \t X Shape Gesture \n"
    "3: \t Twist Counter Clockwise Gesture \n"
    "4: \t Circle Vertical Counter Clockwise \n"
)

the prediction is: 	  [3 3 0 4 2 1 3 3 1 1 0 2 1 2 0 0 1 2 1 4]
the actual is: 		  [3 3 0 4 2 1 2 3 1 1 0 2 1 2 0 0 2 2 1 4]


0: 	 Circle Horizontal Counter Clockwise Gesture 
1: 	 V Shape Gesture 
2: 	 X Shape Gesture 
3: 	 Twist Counter Clockwise Gesture 
4: 	 Circle Vertical Counter Clockwise 



In [439]:
# FINISHED!